# Dynamic Rate Limiting — Adaptive API Protection in Spring Boot

## Overview

This trimester, I implemented a dynamic rate limiting system in the Spring Boot backend to protect APIs from abuse while maintaining fairness and scalability under varying load conditions.

Rather than using a static per-user request limit, I designed a system that:

- Tracks active authenticated users  
- Dynamically adjusts request limits based on system load  
- Rebuilds user-specific token buckets when limits change  
- Uses Bucket4j for in-memory token bucket enforcement  
- Prevents abuse while scaling gracefully  

The result is an adaptive rate limiter that balances protection and performance.

---

## Part 1 — The Problem

APIs are vulnerable to:

- Brute-force login attempts  
- API scraping  
- Automated abuse  
- Resource exhaustion under high load  

A static rate limit (e.g., 20 requests per minute) has drawbacks:

- Too restrictive when few users are active  
- Too permissive during high traffic  
- Does not scale with system load  

We needed a smarter system that adjusts limits dynamically.

---

## Part 2 — The Solution

I implemented a `RateLimitFilter` using Spring's `OncePerRequestFilter` and Bucket4j.

```java
@Component
public class RateLimitFilter extends OncePerRequestFilter {
```

This filter:

- Executes once per HTTP request  
- Identifies the user (or IP if unauthenticated)  
- Computes a dynamic rate limit  
- Applies token bucket enforcement  
- Returns HTTP 429 when the limit is exceeded  

---

## Part 3 — Core Design

### 1. User Identification

Each request is mapped to a unique key:

```java
String username = request.getUserPrincipal() != null
        ? request.getUserPrincipal().getName()
        : request.getRemoteAddr();
```

This ensures:

- Authenticated users are rate limited individually  
- Unauthenticated users are rate limited by IP  

---

### 2. Active User Tracking

Authenticated users are tracked:

```java
if (request.getUserPrincipal() != null) {
    activeUsers.put(username, true);
}
```

This allows the system to compute limits based on real-time user load.

---

### 3. Dynamic Limit Calculation

Instead of a fixed limit, the system adjusts based on active users:

```java
private int computeDynamicLimit(String username, int activeUsers) {
    if (activeUsers < 5) {
        return 100;
    } else if (activeUsers < 20) {
        return 300;
    } else {
        return 600;
    }
}
```

#### Logic Breakdown

| Active Users | Requests Per Minute |
|-------------|--------------------|
| < 5         | 100                |
| < 20        | 300                |
| 20+         | 600                |

This ensures:

- Generous limits when few users are online  
- Higher aggregate throughput when system usage grows  
- Adaptive scaling behavior  

---

## Part 4 — Token Bucket Implementation (Bucket4j)

Each user has their own bucket stored in a concurrent cache:

```java
private final Map<String, Bucket> cache = new ConcurrentHashMap<>();
private final Map<String, Integer> userLimits = new ConcurrentHashMap<>();
```

When a limit changes, the bucket is rebuilt:

```java
cache.put(username, createBucketWithLimit(dynamicLimit));
userLimits.put(username, dynamicLimit);
```

---

### Bucket Creation

```java
private Bucket createBucketWithLimit(int limit) {
    Bandwidth bandwidth = Bandwidth.builder()
            .capacity(limit)
            .refillGreedy(limit, Duration.ofMinutes(1))
            .build();

    return Bucket.builder()
            .addLimit(bandwidth)
            .build();
}
```

#### How This Works

- `capacity(limit)` → Maximum tokens available  
- `refillGreedy(limit, Duration.ofMinutes(1))` → Refill all tokens every minute  
- `tryConsume(1)` → Consumes 1 token per request  

If no tokens remain, the request is rejected.

---

## Part 5 — Request Flow

For every request:

1. Identify user or IP  
2. Track active authenticated users  
3. Compute dynamic limit  
4. Rebuild bucket if limit changed  
5. Attempt to consume 1 token  
6. If allowed → continue request  
7. If denied → return HTTP 429  

```java
if (bucket.tryConsume(1)) {
    filterChain.doFilter(request, response);
} else {
    response.setStatus(429);
    response.getWriter().write("Too many requests - limit: " + dynamicLimit + " per minute");
}
```

---

## Part 6 — Why This Design Works

### Thread-Safe

Uses `ConcurrentHashMap` for safe concurrent access.

### Per-User Isolation

Each user/IP has their own independent bucket.

### Dynamic Adaptation

Limits adjust automatically as user load changes.

### Minimal Overhead

In-memory token buckets are lightweight and fast.

### Configurable Defaults

Default limit can be injected via:

```java
@Value("${security.rate-limit.requests-per-minute:20}")
```

This allows environment-based configuration.

---

## Part 7 — Security Impact

This rate limiter protects against:

- Brute force attacks  
- API scraping  
- Request flooding  
- Denial-of-service style abuse  

And it does so without punishing legitimate users during low-traffic periods.

---

## Future Improvements

### Near-Term

- Add sliding window tracking instead of greedy refill  
- Track burst detection patterns  
- Add logging for rate-limit violations  
- Include `Retry-After` header  

### Longer-Term

- Redis-backed distributed rate limiting  
- Separate limits per endpoint  
- Role-based rate limits (admin vs student)  
- Integration with anomaly detection system  
- Dashboard for monitoring rate-limit metrics  

---

## Reflection

This project reinforced an important engineering principle:

> Security controls should scale with system behavior.

A fixed rate limit is simple, but adaptive rate limiting is resilient.

By combining:

- Spring Security  
- Bucket4j token buckets  
- Real-time active user tracking  

I built a rate limiting system that protects APIs while preserving flexibility.

It is not just a throttle — it is an adaptive protection layer.
